In [ ]:
# App Rating Prediction

## Dataset
# App: Application name
# Category: Category of the app
# Rating: Overall user rating of the app (target variable)
# Reviews: Number of user reviews
# Size: Size of the app
# Installs: Number of user downloads/installs
# Type: Paid or Free
# Price: Price of the app
# Content Rating: Age group the app is targeted at (Children, Mature 21+, Adult)
# Genres: Additional genres the app belongs to
# Last Updated: Date when the app was last updated
# Current Ver: Current version of the app
# Android Ver: Minimum required Android version


## Coding
# connecting, uploading data, uploading or creating a new notebook,
# importing liabraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Reading the data
df = pd.read_csv('abcplaystore.csv')
df.head(30)

#Data profiling
# Checking and standardizing data types
# View data format
df.info()

# Standardizing data format
# Define the function
def convert_Size(Size_str):
    if pd.isna(Size_str):
        return None
    Size_str = Size_str.replace(',', '')  # Remove commas
    if any(char in Size_str for char in ['K', 'M', '+']):
        return float(Size_str.replace('K', '').replace('M', '').replace('+', ''))
    elif Size_str.isdigit():
        return float(Size_str)
    else:
        return None
# Apply convert_Size function to Size column
df['Size_numeric'] = df['Size'].apply(convert_Size)

def convert_Installs(Installs_str):
    if pd.isna(Installs_str):  # Check if the value is NaN
        return None
    Installs_str = Installs_str.replace(',', '')  # Remove commas
    if '+' in Installs_str:  # Check if there's a plus sign
        return float(Installs_str.replace('+', ''))
    elif Installs_str.isdigit():  # Check if the string is a digit (after comma removal)
        return float(Installs_str)
    else:  # If neither condition matches, return None
        return None

df['Installs_numeric'] = df['Installs'].apply(convert_Installs)

def convert_Price(Price_str):
    if pd.isna(Price_str):
        return None
    Price_str = Price_str.replace(',', '')  # Remove commas
    if '$' in Price_str:
        return float(Price_str.replace('$', ''))
    elif Price_str.isdigit():
        return float(Price_str)
    else:
        return None
df['Price_numeric'] = df['Price'].apply(convert_Price)

df['Reviews_numeric'] = pd.to_numeric(df['Reviews'], errors='coerce')

df.info()

# Data cleaning
# Remove rows with missing values, if too many are deleted, it indicates some numbers are 0 and were missed
df.shape

df.isnull().sum()/df.shape[0]*100
df_cleaned = df.dropna()
df_cleaned.isnull().sum()

df_cleaned.shape

# Remove duplicate rows
# Count duplicate rows
df_cleaned.duplicated().sum()
# Print and view duplicate rows
identical_rows = df[df.duplicated(keep=False)]
# Sort and print duplicate rows
identical_rows_sorted = identical_rows.sort_values(by=list(df.columns))
print(identical_rows_sorted)
# Remove duplicate rows
df_cleaned.drop_duplicates(inplace=True)
df_cleaned.shape

# Remove logically incorrect rows
df_cleaned = df_cleaned[(df_cleaned['Rating'] >= 1) & (df_cleaned['Rating'] <= 5)]
df_cleaned = df_cleaned[df_cleaned['Reviews_numeric'] <= df_cleaned['Installs_numeric']]
df_cleaned = df_cleaned[~((df_cleaned['Type'] == 'Free') & (df_cleaned['Price_numeric'] > 0))]
df_cleaned.shape

# Rename and save new table
# Drop the old columns
df_cleaned = df_cleaned.drop(columns=['Price', 'Size', 'Installs', 'Reviews'])

# Rename the new columns to the original names
df_cleaned = df_cleaned.rename(columns={
    'Price_numeric': 'Price',
    'Size_numeric': 'Size',
    'Installs_numeric': 'Installs',
    'Reviews_numeric': 'Reviews'  # Corrected colon issue here
})

# Adjust column order, group numeric columns together
rating_col = df_cleaned.pop('Rating')  # Remove 'Rating' column and save it
df_cleaned['Rating'] = rating_col      # Re-add it at the end

df_cleaned.to_excel('df_cleaned.xlsx')

#make sure the data is properly cleaneddf_cleaned.head()
df_cleaned.head()
df_cleaned.info()
df_cleaned.describe()
df_cleaned.isnull().sum()

# Univariate analysis, removing outliers — actually, it's clearer in Tableau

# Using boxplot and histograms to check for outliers
plt.hist(df_cleaned['Rating'])

plt.hist(df_cleaned['Size'])

sns.boxplot(df_cleaned['Price'])
plt.hist(df_cleaned['Price'])

sns.boxplot(df_cleaned['Reviews'])
plt.hist(df_cleaned['Reviews'])

da = df_cleaned['Installs'].value_counts()
da = da.sort_index()
da

da.index = da.index.astype(str)

# Plotting the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=da.index, y=da.values, color='skyblue')
plt.xlabel('Installs (integer)')
plt.ylabel('Count')
plt.title('Count of Unique Installs')
plt.xticks(rotation=90)  # Rotate x labels for better readability
plt.show()

import numpy as np

percentiles = np.percentile(df_cleaned['Installs'], [10, 25, 50, 70, 90, 95, 99])
percentiles

# Based on the analysis (Tableau), decide which outliers to remove
df_cleaned = df_cleaned.drop(df_cleaned[df_cleaned['Price'] > 200].index)
df_cleaned = df_cleaned.drop(df_cleaned[df_cleaned['Reviews'] > 2000000].index)
df_cleaned = df_cleaned.drop(df_cleaned[df_cleaned['Installs'] >= 5.e+07].index)

sns.boxplot(df_cleaned['Installs'])

# Bivariate relationship analysis - Python seems better
sns.boxplot(df_cleaned['Reviews']/df_cleaned['Installs']*100, palette="Set3")
plt.xlabel('Ratio of Reviews to Installs (%)')
plt.title('Boxplot of Ratio of Reviews to Installs')

# Scatter plot between numeric variables - checking for positive or negative correlation
sns.jointplot(x=df_cleaned['Price'], y=df_cleaned['Rating'], kind='scatter')
sns.jointplot(x=df_cleaned['Size'], y=df_cleaned['Rating'], kind='scatter')
sns.jointplot(x=df_cleaned['Reviews'], y=df_cleaned['Rating'], kind='scatter')

# Boxplot between categorical and numeric variables - checking for significant differences
sns.boxplot(x=df_cleaned['Content Rating'], y=df_cleaned['Rating'])
sns.boxplot(x=df_cleaned['Category'], y=df_cleaned['Rating'])

inp1 = df_cleaned
inp1.shape

# Change 'Review' to 'Reviews' to access the correct column
inp1['Reviews_log'] = np.log1p(inp1['Reviews'])
inp1['Installs_log'] = np.log1p(inp1['Installs'])

inp1.drop(['App','Last Updated','Current Ver','Android Ver'], axis=1, inplace=True)

inp1.shape

inp1 = pd.get_dummies(inp1, columns=['Category', 'Genres', 'Content Rating'])

inp1 = pd.get_dummies(inp1, columns=['Type'])

inp1

# Regression
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(inp1, test_size=0.3, random_state=42)

# Features
X_train = df_train.drop('Rating', axis=1)
X_test = df_test.drop('Rating', axis=1)

# Target variable
y_train = df_train['Rating']
y_test = df_test['Rating']

from sklearn import linear_model

# Initialize linear regression model
model = linear_model.LinearRegression()

# Train the model on the training set
model.fit(X_train, y_train)

# Print model coefficients
print("Model Coefficients:")
print(model.coef_)

from sklearn.metrics import r2_score
y_pred_train = model.predict(X_train)
r2_train = r2_score(y_train, y_pred_train)
print(f"R2 score on the training set: {r2_train}")

comparison_df = pd.DataFrame({'Actual': y_train, 'Predicted': y_pred_train})

# Print comparison results and R2 score on the training set
print("Training set prediction comparison:")
print(comparison_df)
print("\nR2 score on the training set:", r2_train)
